# Secretary Module - Interactive Testing

**Run cells in order from top to bottom!**

Setup takes ~5 minutes on first run.

## 1. Setup - Clone Repo

In [ ]:
import os

if not os.path.exists('/content/ESTA-Final'):
    print("Cloning repository...")
    !git clone https://github.com/JPmarsxxxi/ESTA-Final.git
else:
    print("Pulling latest changes...")
    !cd /content/ESTA-Final && git pull origin claude/secretary-module-orchestration-LPq69

!cd /content/ESTA-Final && git checkout claude/secretary-module-orchestration-LPq69
%cd /content/ESTA-Final/workers/secretary

print("\n✅ Repository ready!")
!ls -la

## 2. Install Ollama + Mistral

**Takes 3-5 minutes (includes 4GB model download)**

In [ ]:
# Install system dependencies
!apt-get update -qq && apt-get install -y -qq zstd curl

# Install Ollama
!curl -fsSL https://ollama.com/install.sh | sh

# Start Ollama server
import subprocess, time, os

if os.path.exists('/usr/local/bin/ollama') or os.path.exists('/usr/bin/ollama'):
    ollama_process = subprocess.Popen(['ollama', 'serve'], stdout=subprocess.PIPE, stderr=subprocess.PIPE)
    time.sleep(5)
    print("✅ Ollama server started")
    
    # Pull Mistral model
    !ollama pull mistral
    !curl -s http://localhost:11434/api/tags | head -20
    print("\n✅ Ollama + Mistral ready!")
else:
    print("❌ Ollama installation failed")

## 3. Install TTS + Whisper

**Takes 1-2 minutes**

In [ ]:
!pip install -q TTS openai-whisper requests
print("✅ TTS, Whisper, and dependencies installed!")

## 4. Import Secretary Module

In [ ]:
from secretary import Secretary, create_full_workflow
from tool_registry import AVAILABLE_TOOLS, REAL_SCRIPTWRITER_AVAILABLE, REAL_AUDIOAGENT_AVAILABLE
import json

print("✅ Secretary loaded!")
print(f"\n🔧 Worker Status:")
print(f"   ScriptWriter: {'REAL ✅' if REAL_SCRIPTWRITER_AVAILABLE else 'MOCK ⚠️'}")
print(f"   AudioAgent: {'REAL ✅' if REAL_AUDIOAGENT_AVAILABLE else 'MOCK ⚠️'}")

if REAL_SCRIPTWRITER_AVAILABLE and REAL_AUDIOAGENT_AVAILABLE:
    print("\n✅ All workers ready!")
else:
    print("\n⚠️  Some workers not available - check installations above")

## 5. Input Requirements

In [ ]:
print("="*60)
print("VIDEO REQUIREMENTS")
print("="*60)

topic = input("\n1. Video topic (min 5 chars): ")
print("\n2. Style: funny, documentary, serious, graphic-heavy, tutorial, or custom")
style = input("   Your choice: ")
print("\n3. Duration (e.g., '5 minutes', 'like 30 seconds', 'around 2-3 mins')")
duration = input("   Duration: ")
audio = input("\n4. Audio mode (generate/upload): ")
script = input("\n5. Script mode (generate/upload): ")
comments = input("\n6. Comments (optional, press Enter to skip): ")

requirements = {
    'topic': topic,
    'style': style,
    'duration_range': duration,
    'audio_mode': audio,
    'script_mode': script,
    'comments': comments or None
}

print("\n" + "="*60)
print(json.dumps(requirements, indent=2))

## 6. Validate Requirements

In [ ]:
sec = Secretary()
is_valid, errors = sec.validate_inputs(requirements)

print("\n" + "="*60)
print("VALIDATION RESULT")
print("="*60)

if is_valid:
    print("✅ ALL INPUTS VALID!")
    sec.set_requirements(requirements)
else:
    print("❌ VALIDATION FAILED!\n")
    for error in errors:
        print(f"  - {error}")
    print("\n⚠️  Fix errors and re-run cell 5!")

## 7. Generate Script (Ollama + LangSearch)

**Takes 30-90 seconds**

In [ ]:
print("🚀 Generating script...\n")

result = sec.call_tool('scriptwriter', sec.requirements)

if result['success']:
    print("\n" + "="*80)
    print("✅ SCRIPT GENERATED!")
    print("="*80)
    
    outputs = result['outputs']
    metadata = result.get('metadata', {})
    
    print(f"\n📊 Stats: {metadata.get('actual_words', 0)} words, ~{outputs.get('estimated_duration_minutes', 0)} min")
    
    sources = outputs.get('research_sources', [])
    if sources:
        print(f"\n📚 Research: {len(sources)} sources")
    
    print(f"\n💡 Talking points: {outputs.get('talking_points_count', 0)}")
    
    print("\n" + "="*80)
    print("📜 SCRIPT:")
    print("="*80)
    print(outputs['script'])
    print("="*80)
    
    script = outputs['script']
    print("\n✅ Ready for audio generation!")
else:
    print(f"\n❌ FAILED: {result.get('error', 'Unknown error')}")

## 8. Generate Audio (XTTS + Whisper)

**Takes 1-3 minutes**

In [ ]:
print("🎤 Generating audio...\n")

style = sec.requirements.get('style', 'neutral')
result = sec.call_tool('audio_agent', {'script': script, 'style': style})

if result['success']:
    print("\n" + "="*80)
    print("✅ AUDIO GENERATED!")
    print("="*80)
    
    outputs = result['outputs']
    
    print(f"\n📊 Duration: {outputs['duration']:.1f}s ({outputs['duration']/60:.1f} min)")
    print(f"📁 File: {outputs['audio_path']}")
    print(f"💾 Size: {outputs['file_size_mb']:.2f} MB")
    
    transcript = outputs['transcript_timestamps']
    segments = transcript['segments']
    
    print(f"\n📝 Segments: {len(segments)}")
    print(f"🔇 Silence points: {len(transcript.get('silence_points', []))}")
    
    print("\n🎯 First 5 segments:")
    for i, seg in enumerate(segments[:5], 1):
        print(f"   [{seg['start']:.1f}s-{seg['end']:.1f}s] {seg['type'].upper()}: {seg['text'][:60]}...")
    
    audio_file = outputs['audio_file']
    print("\n✅ Audio ready! Download from above path.")
else:
    print(f"\n❌ FAILED: {result.get('error', 'Unknown error')}")

## 9. LangSearch (Mock)

Coming next...

In [ ]:
print("⚠️  LangSearch worker not yet implemented - using mock")

result = sec.call_tool('langsearch', {'script': script})

if result['success']:
    research_data = result['outputs']['research_data']
    print(f"✅ Mock research complete: {research_data['total_terms_found']} terms")
else:
    print(f"❌ FAILED: {result.get('error')}")

## Other Workers (Mocks)

BrainBox, AssetCollector, Executor - coming soon!